## Local Function Review

In [15]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)      
    return 

In [16]:
#test 
outer_function()

In [8]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
        
    return inner_function

In [9]:
my_func = outer_function


In [10]:
my_func()


<function __main__.outer_function.<locals>.inner_function>

In [11]:
def outer_function(msg):
    message = msg
    def inner_function():
        print(message)        
    return inner_function

In [12]:
hi_func = outer_function("Hi")
by_func = outer_function("Bye")

hi_func()
by_func()

Hi
Bye


In [17]:
def outer_function(msg):
    def inner_function():
        print(msg)        
    return inner_function

In [18]:
hi_func = outer_function("Hi")
by_func = outer_function("Bye")

hi_func()
by_func()

Hi
Bye


## Decorators
- A function that takes another function as argument
- Add some functionality
- Returns another function
- All of this without altering the orginal function

In [19]:
def decorator_function(original_function):
    def wrapper_function():
        return original_function()
    # Return function
    return wrapper_function

def display():
    print("Display function ran")

In [20]:
# test
decorated_display = decorator_function(display)
decorated_display

<function __main__.decorator_function.<locals>.wrapper_function>

In [21]:
decorated_display()

Display function ran


In [22]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(
        original_function.__name__))
        return original_function()
    # Return function
    return wrapper_function

def display():
    print("Display function ran")

In [23]:
decorated_display = decorator_function(display)
decorated_display()

Wrapper executed this before display
Display function ran


## Now decorate your functions
**@Decorator_function_name

In [24]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(
        original_function.__name__))
        return original_function()
    # Return function
    return wrapper_function

@decorator_function
def display():
    print("Display function ran")

In [25]:
# test
display()

Wrapper executed this before display
Display function ran


This will not work if our original function take arguments

In [29]:
@decorator_function
def display_info(name,age):
    print("display_info ran with arguments ({},{})".format(name,age))
    


display_info("mario", 21)

need to use \*arg and \*\*kwargs

In [31]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print("Wrapper executed this before {}".format(
        original_function.__name__))
        return original_function(*args, **kwargs)
    # Return function
    return wrapper_function

@decorator_function
def display():
    print("Display function ran")
    
@decorator_function
def display_info(name,age):
    print("display_info ran with arguments ({},{})".format(name,age))

In [32]:
display()


Wrapper executed this before display
Display function ran


In [33]:
display_info("Mario",21)

Wrapper executed this before display_info
display_info ran with arguments (Mario,21)


### Example: escape unicode


In [37]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return ascii(x)
    return wrap

@escape_unicode
def mexico_city():
    # Alt + 130 é (nmu-pad)
    return "México"

In [38]:
mexico_city()


"'M\\xe9xico'"

### what can be a Decorator?
- Class Objects, which can be callable with the **dunder call **
- functions as decorators
- Must be callable object

## Classes as Decorators

In [45]:
class DecoratorClass(object):
    def __init__(self, original_function):
        self.original_function = original_function
        
    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}".format(
        self.original_function.__name__))
        return self.original_function(*args, **kwargs) 
    
@DecoratorClass
def display():
    print("Display function ran")
    
@DecoratorClass
def display_info(name,age):
    print("display_info ran with arguments ({},{})".format(name,age))

In [46]:
#test
display()
display_info("Mario",21)

Call method executed this before display
Display function ran
Call method executed this before display_info
display_info ran with arguments (Mario,21)


### Instances as Decorators
A Class Instance

In [50]:
class Trace:
    def __init__(self):
        self._enable = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}".format(f.__name__))
            return f(*args, **kwargs)
        return wrap
    
# create an instance
tracer = Trace()
# instance as decorator
@tracer
def rotate_list(l):
    return l[1:] + [l[0]]



Unlike our previous example the **class object itself is not the decorator**.  rather, instance of trace can be use as decorators.

In [54]:
#test
l1 =[1,2,3]
l1 = rotate_list(l1)
print(l1)

Calling rotate_list
[2, 3, 1]


In [53]:
tracer._enabled = False
l1 =[1,2,3]
l1 = rotate_list(l1)
print(l1)

Calling rotate_list
[2, 3, 1]


## back to function decorators

In [55]:
import time

def my_timer(orginal_func):
    """
    Help you keep track of the loggin part of a function
    """
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orginal_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(orginal_func.__name__, t2))
        return result
    
    return wrapper

#test it
@my_timer
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({},{})".format(name,age))
    
    

In [56]:
#test
display_info("mario",21)

display_info ran with arguments (mario,21)
display_info ran in: 1.002415657043457


## multiple decorators

In [65]:
def my_logger(orignal_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        orignal_func.__name__), level=logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}, and kwargs {}".format(args, kwargs))
        return orignal_func(*args, **kwargs)
    return  wrapper

@my_logger
def display_info(name,age):
    print("display_info ran with arguments ({},{})".format(name,age))

In [66]:
display_info("Weber",21)

display_info ran with arguments (Weber,21)


# test with multile decorators

@my_logger
@my_timer
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({},{})".format(name,age))
    
display_info("Weber",21)

wrap everythink

In [79]:
import time
from functools import wraps

def my_logger(orignal_func):
    import logging
    logging.basicConfig(filename='{}1.log'.format(
        orignal_func.__name__), level=logging.INFO)
    
    @wraps(orignal_func)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}, and kwargs {}".format(args, kwargs))
        return orignal_func(*args, **kwargs)
    return  wrapper

def my_timer(orginal_func):
    """
    Help you keep track of the loggin part of a function
    """
    @wraps((orginal_func))
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orginal_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(orginal_func.__name__, t2))
        return result
    
    return wrapper

@my_logger
@my_timer
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({},{})".format(name,age))
    
display_info("Weber",21)

display_info ran with arguments (Weber,21)
display_info ran in: 1.0001859664916992
